In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Iterator, Tuple
import random
import unittest

import numpy as np

# Square Object (dataclass used in SubSquareSolver)

In [2]:
@dataclass(frozen=True)
class Square:
    """
    A data class representing a square matrix.

    This class encapsulates a 2D square matrix (a NumPy ndarray) and provides
    additional functionality and validations specific to square matrices.

    Attributes:
    matrix (np.ndarray): A 2D NumPy array representing the square matrix. 

    Methods:
    size: Returns the size (length of a side) of the square matrix.

    Raises:
    TypeError: If the input is not a NumPy array.
    ValueError: If the input matrix is not 2-dimensional, not square (equal number of rows and columns),
                or contains elements other than 0s and 1s.
    """
    matrix: np.ndarray = field(repr=False)  # Avoid showing large array in repr

    def __post_init__(self):
        self._valid_matrix()

    def _valid_matrix(self):
        """Perform validations on the matrix."""
        # Ensure matrix is np.ndarray
        if not isinstance(self.matrix, np.ndarray):
            raise TypeError("Matrix must be a NumPy array.")

        # Ensure matrix is 2d-array
        if self.matrix.ndim != 2:
            raise ValueError("Matrix must be 2-dimensional.")

        # Ensure side lengths equal in square
        rows, cols = self.matrix.shape
        if rows != cols:
            raise ValueError("Matrix must be square (equal number of rows and columns).")

        # Ensure each value in square is 0 | 1
        if not np.all(np.isin(self.matrix, [0, 1])):
            raise ValueError("Matrix elements must be either 0s or 1s.")

    @property
    def size(self) -> int:
        """Return the size (length of a side) of the square."""
        return self.matrix.shape[0]

    def __repr__(self) -> str:
        return f"Square(size={self.size})"

# Generate Square object helper function (not currently used)

In [3]:
def generate_square_with_one_valid_sub_square(square_size: int, sub_square_size: int) -> Square:
    """
    Generate a Square object representing a (square_size x square_size) square matrix with random 0s and 1s,
    which contains at least one sub-square of a specified size where the entire perimeter 
    is composed of 1s.

    Parameters:
    square_size (int): The size of the square matrix to be generated.
    sub_square_size (int): The size of the sub-square whose perimeter is to be set to 1s.
                           This value must be less than or equal to square_size and greater than 1.

    Returns:
    Square: A Square data class instance representing the generated square matrix.

    Raises:
    ValueError: If 'sub_square_size' is greater than 'square_size' or if it's less than 2.
    """
    if sub_square_size > square_size or sub_square_size < 2:
        raise ValueError("Sub-square size must be less than or equal to square_size and greater than 1")

    # Initialize the square with random values (0 or 1)
    matrix = np.random.randint(0, 2, size=(square_size, square_size))

    # When sub_square_size equals square_size, the entire perimeter of the square is set
    if sub_square_size == square_size:
        matrix[0, :] = 1  # Top row
        matrix[-1, :] = 1  # Bottom row
        matrix[:, 0] = 1  # Left column
        matrix[:, -1] = 1  # Right column
    else:
        # Randomly choose a position for the sub-square with perimeter one
        row_start = random.randint(0, square_size - sub_square_size)
        col_start = random.randint(0, square_size - sub_square_size)

        # Set the perimeter of the chosen sub-square to one
        matrix[row_start, col_start:col_start + sub_square_size] = 1  # Top row
        matrix[row_start + sub_square_size - 1, col_start:col_start + sub_square_size] = 1  # Bottom row
        matrix[row_start:row_start + sub_square_size, col_start] = 1  # Left column
        matrix[row_start:row_start + sub_square_size, col_start + sub_square_size - 1] = 1  # Right column

    # Return the square wrapped in the Square data class
    return Square(matrix=matrix)

# SubSquareSolver ABC

In [4]:
class SubSquareDNEError(Exception):
    """Raised when sub-square does not exist in Square"""


class SubSquareSolver(ABC):
    """
    Abstract base class for solving the problem of finding the largest sub-square within a Square object.

    A sub-square is defined as a smaller square within the larger Square object whose perimeter consists entirely of ones. 
    This class serves as a template for classes that implement specific algorithms to find the largest such sub-square.

    Subclasses must implement the find_largest_sub_square method to provide their algorithm for this problem.

    Methods:
    find_largest_sub_square(square: Square): Abstract method to find the largest sub-square.
    """

    @abstractmethod
    def find_largest_sub_square(self, square: Square) -> Tuple[np.ndarray, int]:
        """
        Abstract method to find the largest sub-square within the given Square object.

        Subclasses should implement this method with an algorithm to determine the largest sub-square
        whose perimeter is composed entirely of ones.

        Parameters:
        square (Square): The Square object within which to find the largest sub-square.

        Returns:
        Tuple[np.ndarray, int]: A tuple containing the numpy array of the largest sub-square and its size (side length).

        Raises:
        SubSquareDNEError: If no sub-square with a perimeter of ones is found in the square.
        """
        pass

# SlidingWindowSearch algorithm implementation

In [5]:
class SlidingWindowSearch(SubSquareSolver):
    # TODO: add init with timeout duration parameter

    @staticmethod
    def _get_rolling_windows(square: Square, sub_square_size: int) -> Iterator[np.ndarray]:
        """
        Generate an iterator over all possible sub-squares of a given size within a Square object.

        Parameters:
        square (Square): The input Square object. Its matrix should be a square matrix (n x n).
        sub_square_size (int): The side length of the possible sub-squares to be extracted.

        Returns:
        Iterator[np.ndarray]: An iterator that yields each sub-square of the array.

        Raises:
        ValueError: If 'size' is greater than the side length of the square or less than 1.
        """
        n = square.size

        if sub_square_size > n or sub_square_size < 1:
            raise ValueError("Sub Square Size must be less than or equal to the size of the square and greater than 0")

        for start_row in range(n - sub_square_size + 1):
            for start_col in range(n - sub_square_size + 1):
                yield square.matrix[start_row:start_row + sub_square_size, start_col:start_col + sub_square_size]

    @staticmethod
    def _is_sub_square(sub_square: np.ndarray) -> bool:
        """
        Check if all elements on the perimeter of a given sub-square are equal to one.

        This function takes a possible sub-square (a 2D numpy array) and determines whether all the elements
        on its perimeter are equal to 1 and hence a valid sub-square. The perimeter includes the top row, bottom row,
        left column, and right column of the sub-square.

        Parameters:
        sub_square (np.ndarray): A 2D numpy array representing the sub-square to be checked.
                                 The array should be square (m x m).

        Returns:
        bool: True if all elements on the perimeter of the sub-square are equal to 1, False otherwise.
        """
        # Extract the perimeter of the sub-square
        top_row = sub_square[0, :]
        bottom_row = sub_square[-1, :]
        left_column = sub_square[1:-1, 0]
        right_column = sub_square[1:-1, -1]

        # Combine all perimeter elements into one array
        perimeter = np.concatenate((top_row, bottom_row, left_column, right_column))

        # Check if all elements are equal to 1
        return np.all(perimeter == 1)

    def find_largest_sub_square(self, square: Square) -> Tuple[np.ndarray, int]:
        """
        Find the largest sub-square within a given Square object where the perimeter
        is composed entirely of ones. The algorithm iterates over all possible sub-square sizes,
        starting from the largest possible size, and checks each sub-square to find one with a 
        perimeter of ones.
    
        The algorithm uses a sliding window approach to extract all possible sub-squares of each 
        size and checks their perimeter using the '_is_sub_square' method.
    
        Parameters:
        square (Square): A Square object representing the square matrix to be checked.
    
        Returns:
        Tuple[np.ndarray, int]: A tuple containing the numpy array of the largest sub-square and its size (side length).
    
        Raises:
        SubSquareDNEError: If no sub-square with a perimeter of ones is found in the square.
        """
        if not isinstance(square, Square):
            raise TypeError(f"square parameter must be type Square not {type(square)}")

        m = square.size
        for size in range(m, 0, -1):
            for sub_square in self._get_rolling_windows(square=square, sub_square_size=size):
                if self._is_sub_square(sub_square=sub_square):
                    return sub_square, size

        raise SubSquareDNEError(
            "Square does not contain a sub-square with a perimeter of ones. \n{}".format(square.matrix)
        )


# Testing

## Square Dataclass unittest

In [6]:
class TestSquare(unittest.TestCase):
    def test_init_with_valid_parameters(self):
        """Test initialization with valid matrix"""
        matrix = np.array(
            [[1, 0, 0],
             [0, 1, 0],
             [0, 0, 1]]
        )

        square = Square(
            matrix=matrix
        )

        self.assertIsInstance(square.matrix, np.ndarray)
        self.assertIsNone(np.testing.assert_array_equal(square.matrix, matrix))
        self.assertEqual(square.size, 3)

    def test_init_with_matrix_size_one(self):
        """Test initialization with valid matrix size one"""
        matrix = np.array(
            [[1]]
        )

        square = Square(
            matrix=matrix
        )

        self.assertIsInstance(square.matrix, np.ndarray)
        self.assertIsNone(np.testing.assert_array_equal(square.matrix, matrix))
        self.assertEqual(square.size, 1)

    def test_init_with_none_matrix_raises_type_error(self):
        """Test that initializing with 'None' raises a TypeError"""
        with self.assertRaises(TypeError):
            Square(matrix=None)

    def test_init_with_non_binary_matrix_raises_value_error(self):
        """Test that initializing with a non-binary matrix raises a ValueError"""
        with self.assertRaises(ValueError):
            Square(
                matrix=np.array(
                    [[3, 0, 0],
                     [0, 0, 0],
                     [0, 0, 0]]
                )
            )

    def test_init_with_non_2d_matrix_raises_value_error(self):
        """Test that initializing with a non-2D matrix raises a ValueError"""
        with self.assertRaises(ValueError):
            Square(matrix=np.zeros(shape=(3, 3, 3)))

    def test_init_with_non_square_matrix_raises_value_error(self):
        """Test that initializing with a non-square matrix raises a ValueError"""
        with self.assertRaises(ValueError):
            Square(matrix=np.zeros(shape=(2, 3)))

## SlidingWindowSearch unittest

In [7]:
class TestSlidingWindowSearch(unittest.TestCase):
    def setUp(self):
        self.solver = SlidingWindowSearch()
        self.square_3x3 = Square(
            matrix=np.array(
                [[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]]
            )
        )
        self.square_4x4 = Square(
            matrix=np.array(
                [[1, 1, 1, 1],
                 [1, 0, 0, 1],
                 [1, 0, 0, 1],
                 [1, 1, 1, 1]]
            )
        )

    def test_get_rolling_windows_valid(self):
        """Test that correct number of rolling windows is generated for given Square"""
        windows = list(self.solver._get_rolling_windows(square=self.square_3x3, sub_square_size=2))
        self.assertEqual(len(windows), 4)  # 4 possible 2x2 sub-squares in a 3x3 square

    def test_get_rolling_windows_with_invalid_sub_square_size(self):
        """Test that sub-square size greater than size of square raises ValueError"""
        with self.assertRaises(ValueError):
            list(self.solver._get_rolling_windows(square=self.square_3x3, sub_square_size=4))

    def test_is_sub_square(self):
        """Test that true sub_square is evaluated as True"""
        sub_square = np.array(
            [[1, 1],
             [1, 1]]
        )
        self.assertTrue(self.solver._is_sub_square(sub_square))

    def test_is_not_sub_square(self):
        """Test that false sub_square is evaluated as False"""
        sub_square = np.array(
            [[1, 0],
             [1, 1]]
        )
        self.assertFalse(self.solver._is_sub_square(sub_square))

## SlidingWindowSearch integration test 

In [8]:
class TestSlidingWindowSearchIntegration(unittest.TestCase):
    def setUp(self):
        self.solver = SlidingWindowSearch()

    def test_find_largest_sub_square_invalid_type(self):
        """Test that TypeError raised when Square instance is not passed"""
        square = np.array(
            [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]]
        )

        with self.assertRaises(TypeError):
            self.solver.find_largest_sub_square(square)

    def test_find_largest_valid_sub_square(self):
        """Test that valid sub_square array is correctly identified and has correct size"""
        square = Square(
            matrix=np.array(
                [[0, 1, 1],
                 [0, 1, 1],
                 [1, 1, 1]]
            )
        )
        expected_size = 2
        expected_array = np.array(
            [[1, 1],
             [1, 1]]
        )
        sub_square_array, size = self.solver.find_largest_sub_square(square)
        self.assertEqual(size, expected_size)
        self.assertIsNone(np.testing.assert_array_equal(sub_square_array, expected_array))

    def test_find_largest_sub_square_full_size(self):
        """Test that Square of all one's correctly identified and has correct size"""
        square = Square(
            matrix=np.ones((3, 3))
        )
        expected_size = 3
        expected_array = np.array(
            [[1, 1, 1],
             [1, 1, 1],
             [1, 1, 1]]
        )
        sub_square_array, size = self.solver.find_largest_sub_square(square)
        self.assertEqual(size, expected_size)
        self.assertIsNone(np.testing.assert_array_equal(sub_square_array, expected_array))

    def test_find_largest_sub_square_size_one(self):
        """Test that Square with size 1, one is correctly identified and has correct size"""
        square = Square(
            matrix=np.ones((1, 1))
        )
        expected_size = 1
        expected_array = np.array(
            [[1]]
        )
        sub_square_array, size = self.solver.find_largest_sub_square(square)
        self.assertEqual(size, expected_size)
        self.assertIsNone(np.testing.assert_array_equal(sub_square_array, expected_array))

    def test_find_largest_sub_square_not_existing(self):
        """Test that Square with non-existing sub-square raises DNEError"""
        square = Square(
            matrix=np.array(
                [[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, 0]]
            )
        )
        with self.assertRaises(SubSquareDNEError):
            self.solver.find_largest_sub_square(square)

In [9]:
# argv and exit parameters required to run tests in jupyter notebook
unittest.main(argv=[''], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.024s

OK
